<a href="https://colab.research.google.com/github/jfkiviaho/toml/blob/master/linear_elasticity_with_aug_lag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
Linear elastic isotropic material in tension
    
    div(sigma(u)) + f = 0 in rectangular domain
                   ux = 0 on left boundary
            sigma . n = 0 on bottom boundary
            sigma . n = 0 on top boundary
            sigma . n = t on right boundary

'''
# Load required modules
import tensorflow as tf
import tensorflow.keras as keras

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from scipy.optimize import minimize

import numpy as np
import numpy.linalg as la

from pathlib import Path

from math import ceil
from functools import reduce

# Set floating point precision in Keras to double
tf.keras.backend.set_floatx('float64')

# Set global random seed
# Note: this is a shortcut for getting reproducible results.
#tf.random.set_seed(1)

# Auxiliary function
def compose2(f, g):
    return lambda x: f(g(x))


In [ ]:
# Define model
class Model(keras.Sequential):
    def __init__(self):
        '''
        Initialize model with parameters, collocation points, and network 
        topology

        '''
        super(Model, self).__init__()

        ########################################################################
        # Equation and discretization parameters
        ########################################################################
        self.K       = 1.0  # Bulk modulus
        self.G       = 1.0  # Shear modulus
        self.L       = 1.0  # length of rectangular domain
        self.h       = 1.0  # height of rectangular domain
        self.sig_app = 1.0  # applied stress

        self.lam = self.K - 2*self.G/3  # 1st Lame parameter
        self.mu  = self.G               # 2nd Lame parameter

        self.num_x_pts = 100
        self.num_y_pts = 100

        ########################################################################
        # Augmented Lagrangian parameters
        ########################################################################
        self.num_cons = 4

        self.max_con_iters  = 10**2  # max. num. of constrained iterations
        self.max_unc_iters  = 30**1  # max. num. of unconstrained iterations

        self.tol_unc        = 1e-7   # loss gradient tolerance
        self.tol_con        = 1e-8   # constraint tolerance

        self.penalty        = 1.0    # penalty parameter
        self.max_penalty    = 1e5    # maximum penalty 
        self.penalty_growth = 2.0    # penalty growth

        self.lambdas  = np.zeros(self.num_cons, dtype=np.float64)  # multipliers

        ########################################################################
        # Unconstrained optimization parameters
        ########################################################################
        # The unconstrained optimization that is performed within the
        # constrained optimization in the Augmented Lagrangian framework will
        # be undertaken by a derivative of the stochastic gradient descent
        # method (SGD). The challenge addressed here is how to batch the
        # multiple datasets (domain and boundary sample points) and how to
        # repeat those datasets so that, during each epoch of training, every
        # batch is seen at least once
        self.optimizer = tf.optimizers.Adam()

        self.num_pts = [
                self.num_x_pts*self.num_y_pts,  # for domain
                self.num_y_pts,                 # for left boundary
                self.num_x_pts,                 # for bottom boundary
                self.num_y_pts,                 # for right boundary
                self.num_x_pts                  # for top boundary
            ]

        self.batch_sizes = [
                0.02*self.num_pts[0],  
                0.50*self.num_pts[1],  
                0.50*self.num_pts[2],  
                0.50*self.num_pts[3],  
                0.50*self.num_pts[4]   
            ]
        #self.batch_sizes = [10, 5, 5, 5, 5]
        self.batch_sizes = list(map(round, self.batch_sizes))
        print(self.batch_sizes)

        num_batches_per_epoch = [
                self.num_pts[0]/self.batch_sizes[0],
                self.num_pts[1]/self.batch_sizes[1],
                self.num_pts[2]/self.batch_sizes[2],
                self.num_pts[3]/self.batch_sizes[3],
                self.num_pts[4]/self.batch_sizes[4]
            ]
        num_batches_per_epoch = list(map(ceil, num_batches_per_epoch))
        print(num_batches_per_epoch)

        max_num_batches  = reduce(max, num_batches_per_epoch)

        self.num_repeats = [
                max_num_batches/num_batches_per_epoch[0],
                max_num_batches/num_batches_per_epoch[1],
                max_num_batches/num_batches_per_epoch[2],
                max_num_batches/num_batches_per_epoch[3],
                max_num_batches/num_batches_per_epoch[4]
            ]
        self.num_repeats = list(map(ceil, self.num_repeats))
        print(self.num_repeats)

        ########################################################################
        # Collocation points
        ########################################################################
        x0 = np.array([0.0])
        x1 = np.array([self.L])
        x = np.linspace(x0, x1, self.num_x_pts)

        y0 = np.array([0.0])
        y1 = np.array([self.h])
        y = np.linspace(y0, y1, self.num_y_pts)

        x_mesh, y_mesh = np.meshgrid(x, y)

        # Compute normalization constants
        self.mu_x = x.mean()
        self.sig_x = x.std()

        self.mu_y = y.mean()
        self.sig_y = y.std()

        self.x_coords = [
                x_mesh.flatten().reshape(-1,1),
                x0*np.ones(self.num_pts[1]).reshape(-1,1),
                np.linspace(x0, x1, self.num_pts[2]),
                x1*np.ones(self.num_pts[3]).reshape(-1,1),
                np.linspace(x0, x1, self.num_pts[4])
            ]
        self.X_coords = list(map(self.normalize_x, self.x_coords))

        self.y_coords = [
                y_mesh.flatten().reshape(-1,1),
                np.linspace(y0, y1, self.num_pts[1]),
                y0*np.ones(self.num_pts[2]).reshape(-1,1),
                np.linspace(y0, y1, self.num_pts[3]),
                y1*np.ones(self.num_pts[4]).reshape(-1,1)
            ]
        self.Y_coords = list(map(self.normalize_y, self.y_coords))

        self.all_pts = tuple(
                zip(
                    map(tf.constant, self.X_coords), 
                    map(tf.constant, self.Y_coords)
                )
            )

       ########################################################################
        # Neural network topology
        ########################################################################
        # Add first layer
        input_layer = keras.layers.Dense(units=2, input_shape=[None,2])
        self.add(input_layer)

        # Add hidden layers
        num_neurons_per_hidden_layer = [20]

        for num_neurons in num_neurons_per_hidden_layer:
            self.add(
                    keras.layers.Dense(
                        units=num_neurons, 
                        activation=tf.nn.tanh
                    )
                )
        
        # Add last layer
        output_layer = keras.layers.Dense(units=2)
        self.add(output_layer)

        ########################################################################
        # Logging
        ########################################################################

        # Optimization history arrays
        self.iters = [0]
        #self.res_hist = [self.eval_res().numpy()]
        #self.bc_hist = [
        #        [self.eval_bnd_cnd(0).numpy()],
        #        [self.eval_bnd_cnd(1).numpy()],
        #        [self.eval_bnd_cnd(2).numpy()],
        #        [self.eval_bnd_cnd(3).numpy()]
        #    ]

        return

    def normalize_x(self, pts):
        return (pts - self.mu_x)/self.sig_x

    def normalize_y(self, pts):
        return (pts - self.mu_y)/self.sig_y

    def make_datasets(self):
        '''
        Make a list of appropriately shuffled, batched, and repeated datasets
        for the domain and boundary points
        '''
        ds_list = [
                tf.data.Dataset.from_tensor_slices(
                        (X, Y)
                    ).shuffle(buf_size).repeat(num_rpt).batch(bat_size)
                for X, Y, buf_size, num_rpt, bat_size in 
                zip(
                        self.X_coords,
                        self.Y_coords,
                        self.num_pts,
                        self.num_repeats,
                        self.batch_sizes
                    )
            ]

        return ds_list

    def eval_strain(self, X, Y):
        '''
        Evaluate strain at given coordinates

        Parameters
        ----------
        X: TensorFlow tensor
            contains N number of normalized x coordinates at which to evalaute
        Y: TensorFlow tensor
            contains N number of normalized y coordinates at which to evalaute
        
        Returns
        -------
        eps: NumPy array
            the strain tensor at each point in a (N, 2, 2)-shape array
        '''
        dXdx = 1.0/self.sig_x
        dYdy = 1.0/self.sig_y

        with tf.GradientTape(persistent=True) as t2:
            t2.watch(X)
            t2.watch(Y)

            uvec = self.__call__(tf.stack([X, Y], axis=2))
            u = tf.slice(uvec, [0,0,0], [-1,-1,1])
            v = tf.slice(uvec, [0,0,1], [-1,-1,1])
        
        # Compute displacement derivatives (gradient)
        u_x = t2.gradient(u, X)*dXdx
        u_y = t2.gradient(u, Y)*dYdy
        v_x = t2.gradient(v, X)*dXdx
        v_y = t2.gradient(v, Y)*dYdy

        # Compute strain from displacement derivatives
        epsxx = 0.5*(u_x + u_x)
        epsyx = 0.5*(v_x + u_y)
        epsxy = 0.5*(u_y + v_x)
        epsyy = 0.5*(v_y + v_y)

        # Stack the strain components so that there's a 2D tensor for each
        # point
        eps = tf.squeeze(
                tf.stack(
                    [
                        tf.stack([epsxx, epsyx], axis=2),
                        tf.stack([epsxy, epsyy], axis=2)
                    ], 
                    axis=3
                )
            )

        return eps

    def eval_stress(self, X, Y):
        '''
        Evaluate stress at given coordinates

        Parameters
        ----------
        x: TensorFlow tensor
            contains N number of normalized x coordinates at which to evalaute
        y: TensorFlow tensor
            contains N number of normalized y coordinates at which to evalaute
        
        Returns
        -------
        sig: NumPy array
            the stress tensor at each point in a (n, 2, 2)-shape array
        '''
        # Evaluate the strain
        eps = self.eval_strain(X, Y)
        if len(eps.shape) == 3:
            epsxx = tf.slice(eps, [0,0,0], [-1,1,1])
            epsyx = tf.slice(eps, [0,1,0], [-1,1,1])
            epsxy = tf.slice(eps, [0,0,1], [-1,1,1])
            epsyy = tf.slice(eps, [0,1,1], [-1,1,1])
        elif len(eps.shape) == 4:
            epsxx = tf.slice(eps, [0,0,0,0], [-1,-1,1,1])
            epsyx = tf.slice(eps, [0,0,1,0], [-1,-1,1,1])
            epsxy = tf.slice(eps, [0,0,0,1], [-1,-1,1,1])
            epsyy = tf.slice(eps, [0,0,1,1], [-1,-1,1,1])
        else:
            error_msg = 'Strain tensor array has incorrect shape.'
            raise ValueError(error_msg)

        # Compute the stress from the strain
        sigxx = self.lam*(epsxx + epsyy) + 2.0*self.mu*epsxx
        sigyx =                      0.0 + 2.0*self.mu*epsyx
        sigxy =                      0.0 + 2.0*self.mu*epsxy
        sigyy = self.lam*(epsxx + epsyy) + 2.0*self.mu*epsyy

        # Stack the stress components so that there's a 2D tensor for each
        # point
        sig = tf.squeeze(
                tf.stack(
                    [
                        tf.stack([sigxx, sigyx], axis=2),
                        tf.stack([sigxy, sigyy], axis=2)
                    ], 
                    axis=3
                )
            )

        return sig

    def eval_res(self, X, Y):
        '''
        Evaluate residual of governing equation

        Parameters
        ----------
        x: TensorFlow tensor
            normalized x coordinates at which to evalaute
        y: TensorFlow tensor
            normalized y coordinates at which to evalaute
        
        Returns
        -------
        res: TensorFlow tensor
            evaluation of residual
        '''
        dXdx = 1.0/self.sig_x
        dYdy = 1.0/self.sig_y

        # Evaluate residual of governing equation
        with tf.GradientTape(persistent=True) as t1:
            t1.watch(X)
            t1.watch(Y)

            # Evaluate the stress
            sig = self.eval_stress(X, Y)
            sigxx = tf.slice(sig, [0,0,0], [-1,1,1])
            sigyx = tf.slice(sig, [0,1,0], [-1,1,1])
            sigxy = tf.slice(sig, [0,0,1], [-1,1,1])
            sigyy = tf.slice(sig, [0,1,1], [-1,1,1])

        # Compute derivatives (divergence) of stress
        sigxx_x = t1.gradient(sigxx, X)*dXdx
        sigxy_y = t1.gradient(sigxy, Y)*dYdy
        sigyx_x = t1.gradient(sigyx, X)*dXdx
        sigyy_y = t1.gradient(sigyy, Y)*dYdy

        # Compute residual
        N = np.product(X.shape)
        resx = (1/N)*tf.square(sigxx_x + sigxy_y)
        resy = (1/N)*tf.square(sigyx_x + sigyy_y)

        res = tf.reduce_sum(resx + resy)
        
        return res

    def eval_bnd_cnd(self, X, Y, k):
        '''
        Evaluate boundary conditions

        Parameters
        ----------
        X: TensorFlow tensor
            normalized x coordinates at which to evalaute
        Y: TensorFlow tensor
            normalized y coordinates at which to evalaute
        k: int
            boundary indicator (LEFT = 0, BOTTOM = 1, RIGHT = 2, TOP = 3)

        Returns
        -------
        bnd_cnd: TensorFlow tensor
            boundary condition violation

        '''
        N = np.product(X.shape)

        # Left fixed-displacement (Dirichlet) boundary condition 
        # u = 0 at x = 0
        if k == 0:
            # Evaluate predicted displacement at boundary points
            uvec = self.__call__(tf.stack([X, Y], axis=2))
            u = tf.slice(uvec, [0,0,0], [-1,1,1])

            # Compute measure of boundary condition violation
            bnd_cnd = (1/N)*tf.reduce_sum(tf.square(u))

        # Top and bottom traction-free (Neumann) boundary conditions
        # sigxy = sigyy = 0 at y = 0
        elif k == 1 or k == 3:
            # Evaluate the stress
            sig = self.eval_stress(X, Y)
            sigxx = tf.slice(sig, [0,0,0], [-1,1,1])
            sigyx = tf.slice(sig, [0,1,0], [-1,1,1])
            sigxy = tf.slice(sig, [0,0,1], [-1,1,1])
            sigyy = tf.slice(sig, [0,1,1], [-1,1,1])

            # Compute measure of boundary condition violation
            bnd_cnd = (1/N)*tf.reduce_sum(
                    tf.square(sigxy) + 
                    tf.square(sigyy)
                )

        # Right non-zero traction (Neumann) boundary condition
        # sigyx = 0, sigxx = sig_app at x = L
        elif k == 2:
            # Evaluate the stress
            sig = self.eval_stress(X, Y)
            sigxx = tf.slice(sig, [0,0,0], [-1,1,1])
            sigyx = tf.slice(sig, [0,1,0], [-1,1,1])
            sigxy = tf.slice(sig, [0,0,1], [-1,1,1])
            sigyy = tf.slice(sig, [0,1,1], [-1,1,1])

            bnd_cnd = (1/N)*tf.reduce_sum(
                    tf.square(sigxx - self.sig_app) + 
                    tf.square(sigyx)
                )

        else:
            error_msg = 'No boundary matching index specified.'
            raise ValueError(error_msg)

        return bnd_cnd

    def eval_loss(self, pts):
        '''
        Evaluate loss function for given set of domain and boundary points

        Parameters
        ----------
        pts: tuple
            tuple containing tuples of TensorFlow tensors containing x, y
            coordinates for the domain + boundaries

        Returns
        -------
        loss: TensorFlow tensor
            loss function evaluation 
        '''
        # Add objective, i.e. residual of governing equations
        # in domain, to the loss function (Augmented Lagrangian)
        X_dom, Y_dom = pts[0]
        loss = self.eval_res(X_dom, Y_dom)

        # Add multiplier terms and penalty terms for
        # constraints, i.e. boundary conditions, to the
        # Augmented Lagrangian
        for k in range(self.num_cons):
            X_bnd, Y_bnd = pts[k+1]
            bnd_cnd = self.eval_bnd_cnd(
                    X_bnd, Y_bnd, k
                )
            loss += self.lambdas[k]*bnd_cnd 
            loss += self.penalty*tf.square(bnd_cnd)

        return loss

    def train(self):
        '''
        Train the neural network using a Augmented Lagrangian approach to the
        optimization of the residual and the enforcement of the boundary
        conditions. The unconstrained optimization in the inner loop of the
        Augment Lagrangian procedure is performed by the Adam optimizer from
        TensorFlow
        
        '''
        # Counters 
        it = 0      # total iterations
        it_con = 0  # constrained iterations
        it_unc = 0  # unconstrained iterations

        self.lambdas = np.zeros(self.num_cons, dtype=np.float64)  # multipliers
        self.penalty = 1.0                                        # penalty 
        
        norm_loss_grad = 1.0  # norm of the loss function gradient

        # Perform constrained optimization using Augmented Lagrangian framework
        while it_con < self.max_con_iters:
            # Perform unconstrained optimization
            it_unc = 0
            while it_unc < self.max_unc_iters:
                # Create datasets
                ds_list = self.make_datasets()

                # Loop over batches in datasets and update parameters using
                # gradient information
                print('Epoch', it_unc+1)
                for i, batches in enumerate(zip(*ds_list)):
                    # Evaluate loss function and gradient
                    with tf.GradientTape() as t:
                        loss = self.eval_loss(batches)

                    loss_grad = t.gradient(loss, self.trainable_variables)

                    print('batch', i, 'loss', loss.numpy())

                    # Check for None-valued components in the gradient. Replace
                    # these with zero-valued tensors
                    for i, comp in enumerate(loss_grad):
                        if comp is None:
                            loss_grad[i] = tf.zeros(
                                    self.trainable_variables[i].shape,
                                    dtype=tf.float64
                                )

                    # Apply optimization step to model parameters
                    self.optimizer.apply_gradients(
                            zip(
                                loss_grad, 
                                self.trainable_variables
                            )
                        )

                # Evaluate norm of loss function gradient and determine
                # whether to exit unconstrained optimization
                with tf.GradientTape() as t:
                    loss = self.eval_loss(self.all_pts)

                loss_grad = t.gradient(loss, self.trainable_variables)

                # Check for None-valued components in the gradient. Replace
                # these with zero-valued tensors
                for i, comp in enumerate(loss_grad):
                    if comp is None:
                        loss_grad[i] = tf.zeros(
                                self.trainable_variables[i].shape,
                                dtype=tf.float64
                            )

                norm_loss_grad = tf.sqrt(
                        reduce(
                            lambda x, y: x + y,
                            map(
                                compose2(tf.reduce_sum, tf.square),
                                loss_grad
                            )
                        )
                    ).numpy()

                res, bcs = self.eval_model()

                print()
                print('res:', res)
                for k, bc in enumerate(bcs):
                    print('bc{}:'.format(k), bc)
                print()
                print()

                if loss.numpy() < self.tol_unc and norm_loss_grad < self.tol_unc:
                    break

                it_unc += 1
                it += 1

            # Evaluate the constraints and determine whether to exit
            # constrained optimization
            res, cons = self.eval_model()

            norm_cons = la.norm(cons)

            print('finished unconstrained optimization')
            print('res:', res)
            for k, bc in enumerate(cons):
                print('bc{}:'.format(k), bc)
            print()
            print()

            if norm_cons < self.tol_con:
                break

            # Update the multipliers and penalty parameter
            self.lambdas += 2.0*self.penalty*cons
            self.penalty = min(self.penalty_growth*self.penalty, self.max_penalty)
            print(self.lambdas)
            print(self.penalty)

            it_con += 1

        return

    def eval_model(self):
        '''
        Evaluate residual over all domain points and boundary condition
        violations over all boundary points
        '''
        X_dom = tf.constant(self.X_coords[0])
        Y_dom = tf.constant(self.Y_coords[0])
        res = model.eval_res(X_dom, Y_dom).numpy()

        bcs = np.empty(self.num_cons, dtype=np.float64)
        for k in range(self.num_cons):
            X_bnd = tf.constant(self.X_coords[k+1])
            Y_bnd = tf.constant(self.Y_coords[k+1])
            bcs[k] = model.eval_bnd_cnd(X_bnd, Y_bnd, k).numpy()

        return res, bcs

    def predict_displacement(self, x, y):
        '''
        Return the stress tensor at each point specified

        One can easily generate the appropriately shaped array of points using
        the meshgrid function from NumPy.

        Parameters
        ----------
        x: NumPy array
            the x coordinates arranged in a (ny, nx)-shape array
        y: NumPy array
            the y coordinates arranged in a (ny, nx)-shape array

        Returns
        -------
        disp: NumPy array
            the displacements arranged in a (nx, ny, 2)-shape array

        '''
        # Extract coordinates and normalize
        X = tf.constant(self.normalize_x(x))
        Y = tf.constant(self.normalize_y(y))

        # Evaluate displacements at specified coordinates
        disp = tf.squeeze(self.__call__(np.stack([X, Y], axis=2)))

        return disp.numpy()

    def predict_stress_and_strain(self, x, y):
        '''
        Return the stress and strain tensors at each point specified

        One can easily generate the appropriately shaped array of points using
        the meshgrid function from NumPy.

        Parameters
        ----------
        x: NumPy array
            the x coordinates arranged in a (ny, nx)-shape array
        y: NumPy array
            the y coordinates arranged in a (ny, nx)-shape array

        Returns
        -------
        sig: NumPy array
            the stress tensor at each point in a (ny, nx, 2, 2)-shape array
        eps: NumPy array
            the strain tensor at each point in a (ny, nx, 2, 2)-shape array
        '''
        # Extract coordinates and normalize
        X = tf.constant(self.normalize_x(x))
        Y = tf.constant(self.normalize_y(y))

        # Evaluate stress and strain
        sig = self.eval_stress(X, Y)
        eps = self.eval_strain(X, Y)
        
        return sig.numpy(), eps.numpy()

In [ ]:
# Solve problem of linear elastic material in tension and plot results
if __name__ == '__main__':
    ############################################################################
    # Train the model and print residual and violations
    ############################################################################
    model = Model()
    model.train()
    res, bcs = model.eval_model()

    print()
    print('Model results')
    print('-------------')
    print('res:', res)
    for k, bc in enumerate(bcs):
        print('bc{}:'.format(k), bc)

    # Create directory for storing plots
    plot_dir = './test/'
    Path(plot_dir).mkdir(exist_ok=True)

    ############################################################################
    # Plot displacements, stresses, strains
    ############################################################################
    # Generate new set of coordinates for plotting results
    nx = 100
    ny = 20
    x0 = np.array([0.0])
    x1 = np.array([model.L])
    y0 = np.array([0.0])
    y1 = np.array([model.h])
    x = np.linspace(x0, x1, nx)
    y = np.linspace(y0, y1, ny)
    X, Y = np.meshgrid(x, y) 

    # Predict the displacements from the model
    disp = model.predict_displacement(X, Y)
    U = disp[:,:,0]
    V = disp[:,:,1]

    # Since the model is not constrained in v, any translation up or down is
    # possible. Remove this bias for plotting
    V = V - V.mean()

    # Predict the stresses and strains from the model
    sig, eps = model.predict_stress_and_strain(X, Y)

    # Plot displacements
    fig, ax = plt.subplots(figsize=(18,6))

    ax.scatter(
            X, Y, 
            color='cornflowerblue',
            label='reference'
        )
    ax.scatter(
            X + U, Y + V, 
            color='orange',
            label='deformed'
        )

    ax.set_xlabel(r'x', fontsize=16)
    ax.set_ylabel(r'y', fontsize=16)
    ax.legend()
    plt.axis('equal')
    fig.savefig(plot_dir + 'disps.png')

    # Plot stresses
    fig, ax = plt.subplots(2, 2, figsize=(18,6))
    plt.subplots_adjust(
            hspace=0.5,  # height of blank space between subplots?
            wspace=0.5   # width of blank space between subplots?
        )

    for k in range(4):
        # Do some binary math to organize plots into a 2 x 2 grid
        i = (k & 1) >> 0
        j = (k & 2) >> 1

        num_lvls = 20
        stress = sig[:,:,i,j]
        lvls = np.linspace(stress.min(), stress.max(), num_lvls)
        cset = ax[i,j].contourf(X, Y, stress, lvls)
        fig.colorbar(cset, ax=ax[i,j])

        ax[i,j].set_xlabel(r'$x$', fontsize=16)
        ax[i,j].set_ylabel(r'$y$', fontsize=16)
        lbl_str = 'xy'
        ax[i,j].set_title(
                r'$\sigma_{{{0}{1}}}$'.format(lbl_str[i], lbl_str[j]), 
                fontsize=16
            )
        
    fig.savefig(plot_dir + 'stresses.png')

    # Plot strains
    fig, ax = plt.subplots(2, 2, figsize=(18,6))
    plt.subplots_adjust(
            hspace=0.5,  # height of blank space between subplots?
            wspace=0.5   # width of blank space between subplots?
        )

    for k in range(4):
        # Do some binary math to organize plots into a 2 x 2 grid
        i = (k & 1) >> 0
        j = (k & 2) >> 1

        num_lvls = 20
        strain = eps[:,:,i,j]
        lvls = np.linspace(strain.min(), strain.max(), num_lvls)
        cset = ax[i,j].contourf(X, Y, strain, lvls)
        fig.colorbar(cset, ax=ax[i,j])

        ax[i,j].set_xlabel(r'$x$', fontsize=16)
        ax[i,j].set_ylabel(r'$y$', fontsize=16)
        lbl_str = 'xy'
        ax[i,j].set_title(
                r'$\epsilon_{{{0}{1}}}$'.format(lbl_str[i], lbl_str[j]), 
                fontsize=16
            )

    fig.savefig(plot_dir + 'strains.png')

    '''
    ############################################################################
    # Plot optimization/training history
    ############################################################################
    # Plot residual history
    fig, ax = plt.subplots(figsize=(8,5))
    ax.loglog(
            model.iters, model.res_hist,
            color='cornflowerblue',
            linewidth=2,
        )
    ax.set_xlabel(r'iteration', fontsize=16)
    ax.set_ylabel(r'residual', fontsize=16)

    fig.savefig(plot_dir + 'residual.png')

    # Plot boundary condition violation histories
    fig, ax = plt.subplots(2, 2, figsize=(18,6))
    plt.subplots_adjust(
            hspace=0.5,  # height of blank space between subplots?
            wspace=0.5   # width of blank space between subplots?
        )

    for k in range(4):
        # Do some binary math to organize plots into a 2 x 2 grid
        i = (k & 1) >> 0
        j = (k & 2) >> 1

        ax[i,j].loglog(
                model.iters, model.bc_hist[k],
                color='cornflowerblue',
                linewidth=2,
            )
        ax[i,j].set_xlabel(r'iteration', fontsize=16)
        ax[i,j].set_ylabel(r'boundary condition {}'.format(k+1), fontsize=16)

    fig.savefig(plot_dir + 'bnd_cnds.png')
    '''


Streaming output truncated to the last 5000 lines.
batch 36 loss 0.0018187448645569058
batch 37 loss 0.001969311785286787
batch 38 loss 0.0018984560245701757
batch 39 loss 0.0017453037066225323
batch 40 loss 0.0018972832156207405
batch 41 loss 0.0018225182187581606
batch 42 loss 0.0017327130530367027
batch 43 loss 0.0016184120802929642
batch 44 loss 0.001544193343250093
batch 45 loss 0.001998804777625863
batch 46 loss 0.0014173311353093214
batch 47 loss 0.00179765245474973
batch 48 loss 0.0018877456715044303
batch 49 loss 0.001965531823475091

res: 0.0008844433674141575
bc0: 0.004200893642244834
bc1: 0.011843426610829622
bc2: 0.024536692526638888
bc3: 0.011697880466585297


Epoch 20
batch 0 loss 0.001835164614467266
batch 1 loss 0.0020398406486512215
batch 2 loss 0.0018480508631938182
batch 3 loss 0.0019267502787261332
batch 4 loss 0.001876328189873162
batch 5 loss 0.0017162408710610292
batch 6 loss 0.0018104511821727869
batch 7 loss 0.0014583427122156197
batch 8 loss 0.001729165214646